In [ ]:
#!pip install -q tensorflow-gpu==2.0.0-rc1
import tensorflow as tf

# Cifar 데이터
- Cifar 데이터 정보 : https://www.cs.toronto.edu/~kriz/cifar.html
-  간단한 CNN으로도 Mnist 데이터 분류는 98~99%의 정확도
-  논문의 복잡한 모델을 이용하기 위해서 Cifar 10이라는 데이터 활용
-  Cifar는 RGB라는 3개의 채널 데이터가 있음.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
print(x_train.shape, y_train.shape)

(50000, 32, 32, 3) (50000, 1)


In [ ]:
y_train[:10]

array([[6],
       [9],
       [9],
       [4],
       [1],
       [1],
       [2],
       [7],
       [8],
       [3]], dtype=uint8)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# 1. VGGNet _ 모듈 사용하여 구조 알아보기

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16 # 모듈 사용

model = VGG16( # 모델 불러오기
    weights=None, 
    include_top=True, 
    classes=10,
    input_shape=(32,32,3)
)

# Expand this cell for the model summary
model.summary() # 모델의 모양

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 47s 30ms/step - loss: 2.3029 - accuracy: 0.0994
Epoch 2/5
1563/1563 [==============================] - 47s 30ms/step - loss: 2.3028 - accuracy: 0.0985
Epoch 3/5
1563/1563 [==============================] - 48s 30ms/step - loss: 2.3027 - accuracy: 0.0997
Epoch 4/5
1563/1563 [==============================] - 48s 31ms/step - loss: 2.3028 - accuracy: 0.0987
Epoch 5/5
1563/1563 [==============================] - 48s 31ms/step - loss: 2.3027 - accuracy: 0.0983
313/313 - 2s - loss: 2.3027 - accuracy: 0.1000


[2.3026885986328125, 0.10000000149011612]

# 2. VGGNet _ 모델을 변형하여 구현하기

In [ ]:
inputs = keras.Input(shape=(32, 32, 3)) 
x = inputs
x = layers.Conv2D(64, 3, activation='relu', padding="same")(x)
x = layers.Conv2D(64, 3, activation='relu', padding="same")(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu', padding="same")(x)
x = layers.Conv2D(128, 3, activation='relu', padding="same")(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu', padding="same")(x)
x = layers.Conv2D(256, 3, activation='relu', padding="same")(x)
#x = layers.MaxPooling2D(2)(x)

x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dense(256)(x)
x = layers.Dense(10, activation='softmax')(x)
outputs = x

model = keras.Model(inputs, outputs)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)        

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 13s 8ms/step - loss: 1.5560 - accuracy: 0.4259
Epoch 2/5
1563/1563 [==============================] - 12s 8ms/step - loss: 1.0639 - accuracy: 0.6208
Epoch 3/5
1563/1563 [==============================] - 13s 8ms/step - loss: 0.8822 - accuracy: 0.6904
Epoch 4/5
1563/1563 [==============================] - 13s 8ms/step - loss: 0.7609 - accuracy: 0.7351
Epoch 5/5
1563/1563 [==============================] - 13s 8ms/step - loss: 0.6681 - accuracy: 0.7668
313/313 - 1s - loss: 0.7884 - accuracy: 0.7360


[0.7884461879730225, 0.7360000014305115]